In [2]:
import tkinter as tk
from tkinter import messagebox
import socket
import hashlib

In [ ]:
client_socket=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
addr=('localhost',65432)
client_socket.connect(addr)

#hash func
def hashSHA1(password, n): #byte
    data = password.encode('utf-8')
    for i in range(n):
        hash_sha1 = hashlib.sha1()
        hash_sha1.update(data)
        data = hash_sha1.digest()
    return data  



def sendToServer(username,password, instruction):
    try:
        #register=r
        if instruction == 'r':
            D='UserName'+ username
            password = hashSHA1(password, 5)
            password= password.hex()
            D2='Password'+password
            client_socket.sendall(D.encode())
            Udata=client_socket.recv(1024)
            print(Udata.decode())
            client_socket.sendall(D2.encode())
            Pdata=client_socket.recv(1024)
            print(Pdata.decode())
            return Pdata.decode()
        #login =l
        #get number n by username and hash password n-1 times and sent to server
        if instruction == 'l':
            D='UserNumber'+ username
            client_socket.sendall(D.encode())
            n=int(client_socket.recv(1024).decode())
            print(n)
            password = hashSHA1(password, (n-1))
            password= password.hex()
            D2='Password'+password
            client_socket.sendall(D2.encode())
            Pdata=client_socket.recv(1024)
            print(Pdata.decode())
            return Pdata.decode()

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

        

def enter(login,entry_username, entry_password):
    username = entry_username.get()
    password = entry_password.get()
    #no empty cell
    if username=="" or password=="":
        messagebox.showwarning("Warning","Username or password field is empty.")
    else:
        ans=str(sendToServer(username,password, 'l'))
        #if login was successfull
        if ans.startswith('Correct'):
            messagebox.showinfo("",ans)
            login.destroy()
            client_socket.close()
        #wrong pass or username
        else:
            messagebox.showinfo("","Your password or username is wrong.")

#login page
def Login(main):
    main.destroy()
    login = tk.Tk()
    login.title("Login Form")

    label_username = tk.Label(login, text="Username:")
    label_username.grid(row=0, column=0, padx=10, pady=10)
    entry_username = tk.Entry(login)
    entry_username.grid(row=0, column=1, padx=10, pady=10)

    label_password = tk.Label(login, text="Password:")
    label_password.grid(row=1, column=0, padx=10, pady=10)
    entry_password = tk.Entry(login, show="*")
    entry_password.grid(row=1, column=1, padx=10, pady=10)

    enter_button = tk.Button(login, text="Enter", command=lambda:enter(login,entry_username,entry_password))
    enter_button.grid(row=2, column=1, padx=10, pady=10)

#register with user and pass
def submit(register,entry_username, entry_password):
    username = entry_username.get()
    password = entry_password.get()
    if username=="" or password=="": #mustn't be empty
        messagebox.showwarning("Warning","Username or password field is empty.")
    else:
        ans=sendToServer(username,password, 'r')
        #register done
        if ans=="Set": 
            messagebox.showinfo("","Register Complete")
            register.destroy()
            client_socket.close()
        #username has been taken
        else: 
            messagebox.showwarning("Warning",ans)

#register page
def register(main):
    main.destroy()
    register = tk.Tk()
    register.title("Register Form")

    label_username = tk.Label(register, text="Username:")
    label_username.grid(row=0, column=0, padx=10, pady=10)
    entry_username = tk.Entry(register)
    entry_username.grid(row=0, column=1, padx=10, pady=10)

    label_password = tk.Label(register, text="Password:")
    label_password.grid(row=1, column=0, padx=10, pady=10)
    entry_password = tk.Entry(register, show="*")  
    entry_password.grid(row=1, column=1, padx=10, pady=10)

    submit_button = tk.Button(register, text="Submit", command=lambda:submit(register,entry_username, entry_password))
    submit_button.grid(row=2, column=0, padx=10, pady=10)



#mainpage to choose between login and register
main = tk.Tk()
main.title("Main Form")
login_button = tk.Button(main, text="Login", command=lambda:Login(main))
login_button.grid(row=0, column=0, padx=10, pady=10)
register_button = tk.Button(main, text="Register", command=lambda:register(main))
register_button.grid(row=0, column=1, padx=10, pady=10)
main.mainloop()